<a href="https://colab.research.google.com/github/aihuasolo/nah/blob/master/fine_turn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install datasets

In [10]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][10]


{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [11]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [12]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,1 star,"I ordered a pool filter from PoolAZ.com. \n\nOne week passed after my credit card was charged but I never received a delivery notification. I finally called, but got their voicemail.... their hours of operation are 9-3 Mon-Fri. Pretty cushy hours for a retail outfit. Not real handy, to say the least.\n\nSo, I sent an email asking for help. I finally got an email response a coupla days later..... not a phone call with an explanation and maybe even an apology, but a very short story about how FedEx had damaged the item and they were sending another filter. They also said they would send delivery confirmation and tracking in two days.\n\nTwo days turned into a week and I got NOTHING. I sent them another email asking for a status on the delivery. What I got was this, \""Hello currently the supplier we get these from is out of stock, We have issued you a credit to your credit card and canceled the order.\""\n\nNo \""Sorry.\"" ...No \""Is there something else we can do for you?\"" ...No \""Let me show you another filter that will suit your needs.\"" \n\nWhat customer service rep in their right mind grunts out a one-line answer? Something is very wrong with this person. I think their skill sets would be better suited to breaking down cardboard boxes at the Wal-Mart Distribution Center.\n\nBasically no customer service. None. They didn't have time for me and they showed me so with their lame and pathetic email communications and their lack of customer care after they botched my order.\n\nI don't believe for one minute that they ever sent the filter in the first place. And then just canceled my order with no possibility of any kind of resolution. WTF?\n\nI called the filter manufacturer to see if this item was still in production or if it was back-ordered. Yes - still in production. No - not back-ordered.\n\nI contacted another online Pool Supplier and got my filter pronto!\n\nThese Bozos obviously don't want this kind of business and it shows in their lack of service. They could have gotten this filter for me, but instead chose not to.\n\nLame. Pathetic. Unacceptable."
1,5 stars,"I had heard wonderful things about The Porch and finally ventured over to Oakland to try the restaurant for a recent Yelp event. However, I enjoyed the event so much that I came back with a friend just two days later. \n\nThe atmosphere of the restaurant is fantastic. Not only is the patio - or should I say, the porch - of the restaurant great but so is the interior. It feels very modern and relaxed at the same time.\n\nI ordered the Bianca Pizza, a delicious concoction made with fig jam, goat cheese, arugula, and black truffle oil. Since pizza often requires a beer, I ordered my favorite beer of all time (that they have on tap) which is Full Pint White Lightning. My friend had already eaten and just ordered three flavors of gelato - vanilla, chocolate, and hazelnut. She said the flavor of the vanilla was just out of this world.\n\nIf you're ever looking for a beautiful and laid back place to enjoy a delicious meal made with local ingredients, this is the place for you. I'm already dreaming of coming back again."
2,2 star,"I don't know if I'm really digging on these hip sushi joints.. I prefer the more 'traditional' places where everyone working the place screams \""IRASSHAIIIIIIII\"" in falsetto when you walk in the door.\n \n-The fried tofu was delicious, I think there were chunks of wasabi in it. \n-I really liked the presentation of the miso, but 1.) it was lukewarm, 2.) the crock was too big for our little table (plus: nowhere to put the lid), 3.) the ladle was as big as the miso spoons themselves (what's the point???) and it kept falling in to the soup as we tried to maneuver the crock around. Points for form, but function...? fail. \n-The spicy tuna was the most satisfying of the bunch. \n-The Philly roll was cream cheese overload... more salmon, please! \n-The Fuji was pleasantly spicy. \n-The Rainbow Roll was disappointing... wh

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
   return tokenizer(examples["text"],padding="max_length",truncation=True)
tokenized_datasets = dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"Disclaimer: Your first visit you will be completely overwhelmed when you walk through the door. Don't worry. They are prepared for that. Someone will hold your hand (figuratively) and baby step you through the process. \n\nIf you dislike your sandwich here, you don't really have anyone to blame but yourself. Pick your bag and start crafting your masterpiece. 10 bag options. Once you agonize over that, vessel decisions. White, wheat, wrapped in lettuce or in a bowl. Hot or cold. Toppings galore. The combinations and possibilities are endless. Well not endless, but I'm not going to sit here and do that math. Numbers. Hard. \n\nOh and a very important note... THEY HAVE THE GOOD ICE HERE. You know what I'm talking about.\n\nThis location is clean. The workers are ridiculously nice. They pretty much have to be to walk customers through How to Order a Sandwich 101. Parking can be a problem here. You may have to walk a mile... that is if you can even find a spot. \n\nWould you describe yourself as someone who has trouble making decisions? This may not be your best option to grab a sandwich.","[101, 14856, 20737, 4027, 131, 2353, 1148, 3143, 1128, 1209, 1129, 2423, 14127, 1165, 1128, 2647, 1194, 1103, 1442, 119, 1790, 112, 189, 3994, 119, 1220, 1132, 4029, 1111, 1115, 119, 6518, 1209, 2080, 1240, 1289, 113, 20497, 13830, 15306, 1193, 114, 1105, 2963, 2585, 1128, 1194, 1103, 1965, 119, 165, 183, 165, 183, 2240, 2087, 1128, 20662, 1240, 14327, 1303, 117, 1128, 1274, 112, 189, 1541, 1138, 2256, 1106, 7338, 1133, 3739, 119, 20984, 1240, 3821, 1105, 1838, 8315, 1158, 1240, 22055, 119, 1275, 3821, 6665, 119, 2857, 1128, 2403, 2605, 3171, 1166, 1115, 117, 5832, 6134, 119, 2061, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [18]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
!pip install accelerate>=0.20.1

In [12]:
import accelerate
from accelerate.state import AcceleratorState, PartialState
from accelerate.utils import DistributedType

In [13]:
pip show accelerate

Name: accelerate
Version: 0.26.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [1]:
import transformers
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [2]:
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_length=False,
half_precision_backend=au

In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.961600,0.931160,0.615000
2,0.676500,0.930859,0.620000
3,0.412500,1.187808,0.622000


TrainOutput(global_step=1875, training_loss=0.7033283640543619, metrics={'train_runtime': 405.2445, 'train_samples_per_second': 37.015, 'train_steps_per_second': 4.627, 'total_flos': 3946772136960000.0, 'train_loss': 0.7033283640543619, 'epoch': 3.0})

In [22]:
!/opt/bin/nvidia-smi

Fri Jan 26 02:48:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0              55W / 400W |   7937MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [23]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.2305063009262085,
 'eval_accuracy': 0.58,
 'eval_runtime': 0.9167,
 'eval_samples_per_second': 109.09,
 'eval_steps_per_second': 14.182,
 'epoch': 3.0}

In [24]:
trainer.save_model(f"{model_dir}/finetuned-trainer")